In [4]:
pip install pandas pyreadstat

Note: you may need to restart the kernel to use updated packages.


In [62]:
import pandas as pd
import os

# File paths to years 2013-2023 for demographic, depression, early childhood and alcohol use surveys 
# from CDC website "National Health and Nutrition Examenation Survey"
file_paths = [
    "P_ECQ_2017-2020-pre.xpt", "P_DPQ_2017-2020-pre.xpt", 
    "P_ALQ_2017-2020-pre.xpt", "P_DEMO_2017-2020-pre.xpt", 
    "DPQ_L_2021-2023.xpt", "ECQ_L_2021-2023.xpt",
    "ALQ_L_2021-2023.xpt", "DEMO_L_2021-2023.xpt"
]

# Load each file into dictionary of DataFrames
dataframes = {}
for file in file_paths:
    key = os.path.splitext(os.path.basename(file))[0]  # Use filename as key
    dataframes[key] = pd.read_sas(file, format="xport")

# Check loaded files
for key, df in dataframes.items():
    print(f"{key}: {df.shape}")


P_ECQ_2017-2020-pre: (5365, 10)
P_DPQ_2017-2020-pre: (8965, 11)
P_ALQ_2017-2020-pre: (8965, 10)
P_DEMO_2017-2020-pre: (15560, 29)
DPQ_L_2021-2023: (6337, 11)
ECQ_L_2021-2023: (3432, 7)
ALQ_L_2021-2023: (6337, 9)
DEMO_L_2021-2023: (11933, 27)


In [42]:
# Merge 2017-2020 files
dpq_2017 = dataframes['P_DPQ_2017-2020-pre']
alq_2017 = dataframes['P_ALQ_2017-2020-pre']
ecq_2017 = dataframes['P_ECQ_2017-2020-pre']
demo_2017 = dataframes['P_DEMO_2017-2020-pre']

  
# Merge using SEQN
merged_2017 = demo_2017.merge(dpq_2017, on="SEQN", how="inner") \
                       .merge(alq_2017, on="SEQN", how="left") \
                       .merge(ecq_2017, on="SEQN", how="left")

print(merged_2017.shape)
print(merged_2017.head())

# Merge 2021-2023 files
dpq_2021 = dataframes['DPQ_L_2021-2023']
alq_2021 = dataframes['ALQ_L_2021-2023']
ecq_2021 = dataframes['ECQ_L_2021-2023']
demo_2021 = dataframes['DEMO_L_2021-2023']


  
# Merge using SEQN
merged_2021 = demo_2021.merge(dpq_2021, on="SEQN", how="inner") \
                       .merge(alq_2021, on="SEQN", how="left") \
                       .merge(ecq_2021, on="SEQN", how="left")

print(merged_2021.shape)
print(merged_2021.head())

(8965, 57)
       SEQN  SDDSRVYR  RIDSTATR  RIAGENDR  RIDAGEYR  RIDAGEMN  RIDRETH1  \
0  109266.0      66.0       2.0       2.0      29.0       NaN       5.0   
1  109271.0      66.0       2.0       1.0      49.0       NaN       3.0   
2  109273.0      66.0       2.0       1.0      36.0       NaN       3.0   
3  109274.0      66.0       2.0       1.0      68.0       NaN       5.0   
4  109282.0      66.0       2.0       1.0      76.0       NaN       3.0   

   RIDRETH3  RIDEXMON  DMDBORN4  ...        ALQ170  ECD010  ECQ020  ECD070A  \
0       6.0       2.0       2.0  ...  5.397605e-79     NaN     NaN      NaN   
1       3.0       2.0       1.0  ...           NaN     NaN     NaN      NaN   
2       3.0       2.0       1.0  ...           NaN     NaN     NaN      NaN   
3       7.0       1.0       1.0  ...  5.397605e-79     NaN     NaN      NaN   
4       3.0       2.0       1.0  ...           NaN     NaN     NaN      NaN   

   ECD070B  ECQ080  ECQ090  WHQ030E  MCQ080E  ECQ150  
0      N

In [63]:
# Combine all year blocks
combined_df = pd.concat([merged_2017, merged_2021], ignore_index=True)

# Check combined dataset
print(combined_df.shape)
print(combined_df.head())


(15302, 68)
       SEQN  SDDSRVYR  RIDSTATR  RIAGENDR  RIDAGEYR  RIDAGEMN  RIDRETH1  \
0  109266.0      66.0       2.0       2.0      29.0       NaN       5.0   
1  109271.0      66.0       2.0       1.0      49.0       NaN       3.0   
2  109273.0      66.0       2.0       1.0      36.0       NaN       3.0   
3  109274.0      66.0       2.0       1.0      68.0       NaN       5.0   
4  109282.0      66.0       2.0       1.0      76.0       NaN       3.0   

   RIDRETH3  RIDEXMON  DMDBORN4  ...  DMQMILIZ  DMDYRUSR  DMDHHSIZ  DMDHRGND  \
0       6.0       2.0       2.0  ...       NaN       NaN       NaN       NaN   
1       3.0       2.0       1.0  ...       NaN       NaN       NaN       NaN   
2       3.0       2.0       1.0  ...       NaN       NaN       NaN       NaN   
3       7.0       1.0       1.0  ...       NaN       NaN       NaN       NaN   
4       3.0       2.0       1.0  ...       NaN       NaN       NaN       NaN   

   DMDHRAGZ  DMDHREDZ  DMDHRMAZ  DMDHSEDZ  WTINT2YR  WTM

In [70]:
# Make original column readable using codebooks
column_mapping = {
    'SEQN': 'Respondent sequence number',
    'SDDSRVYR': 'Data release cycle',
    'RIDSTATR': 'Interview/Examination status',
    'RIAGENDR': 'Gender',
    'RIDAGEYR': 'Age in years at screening',
    'RIDAGEMN': 'Age in months at screening (0-24 months)',
    'RIDRETH1': 'Race/Hispanic origin',
    'RIDRETH3': 'Race/Hispanic origin with NH Asian',
    'RIDEXMON': 'Six-month time period',
    'RIDEXAGM': 'Age in months at exam (0-19 years)',
    'DMQMILIZ': 'Served active duty in US Armed Forces',
    'DMDBORN4': 'Country of birth',
    'DMDYRUSR': 'Length of time in US',
    'DMDEDUC2': 'Education level (Adults 20+)',
    'DMDMARTZ': 'Marital status',
    'RIDEXPRG': 'Pregnancy status at exam',
    'DMDHHSIZ': 'Total number of people in the household',
    'DMDHRGND': 'Household reference person’s gender',
    'DMDHRAGZ': 'Household reference person’s age in years',
    'DMDHREDZ': 'Household reference person’s education level',
    'DMDHRMAZ': 'Household reference person’s marital status',
    'DMDHSEDZ': 'Household reference person’s spouse’s education level',
    'WTINT2YR': 'Full sample 2-year interview weight',
    'WTMEC2YR': 'Full sample 2-year MEC exam weight',
    'SDMVSTRA': 'Masked variance pseudo-stratum',
    'SDMVPSU': 'Masked variance pseudo-PSU',
    'INDFMPIR': 'Ratio of family income to poverty',
    'ALQ111': 'Ever had a drink of any kind of alcohol',
    'ALQ121': 'Past 12 months how often drink alcohol beverage',
    'ALQ130': 'Average number of alcoholic drinks/day in past 12 months',
    'ALQ142': 'Number of days had 4/5 drinks in past 12 months',
    'ALQ270': 'Number of times had 4/5 drinks in 2 hours (past 12 months)',
    'ALQ280': 'Number of times had 8+ drinks in 1 day (past 12 months)',
    'ALQ151': 'Ever had 4/5 or more drinks every day',
    'ALQ170': 'Number of times had 4/5 drinks on occasion (past month)',
    'DPQ010': 'Little interest in doing things',
    'DPQ020': 'Feeling down, depressed, or hopeless',
    'DPQ030': 'Trouble sleeping or sleeping too much',
    'DPQ040': 'Feeling tired or having little energy',
    'DPQ050': 'Poor appetite or overeating',
    'DPQ060': 'Feeling bad about yourself',
    'DPQ070': 'Trouble concentrating on things',
    'DPQ080': 'Moving or speaking slowly or too fast',
    'DPQ090': 'Thought you would be better off dead',
    'DPQ100': 'Difficulty these problems have caused',
}
